# Example: Water discharge data analysis

In [ ]:
%load_ext autoreload

%autoreload 2

%matplotlib inline

In [ ]:
try:
    from oatlib import sensor, oat_utils
except ImportError:
    !pip install oatlib

In [ ]:
import requests
import json
from oatlib import sensor, method, oat_utils
from dateutil import parser
import matplotlib.pyplot as plt
from statistics import stdev

⚒ Load data from csv using OAT

In [ ]:
SENSOR = sensor.Sensor(
    name='procedure_name',
    prop='water-discharge',
    unit='m3/s'
)
SENSOR.ts_from_csv(
    'https://raw.githubusercontent.com/istSOS/workshop/master/oat/data/Q_VED_ISO_20180101000000000000.dat',
    qualitycol=2,
    sep=',',
    skiprows=1
)

⚒ take a look at the series

In [ ]:
SENSOR.ts.head(10)

⚒ Monthly resample using mean() function 

In [ ]:
SENSOR_M = SENSOR.copy()
SENSOR_M.ts = SENSOR.ts.resample("M").mean()

⚒ Plot monthly Q

In [ ]:
SENSOR_M.ts['data'].plot(figsize=(16,5))

## Multi-year monthly means with groupby

⚒ First step will be to add another column to our DataFrame with month numbers:

In [ ]:
SENSOR.ts['mon'] = SENSOR.ts.index.month

⚒ Now we can use [groupby](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.groupby.html) to group our values by months and calculate mean for each of the groups (month in our case):

In [ ]:
monmean = SENSOR.ts.groupby('mon').aggregate('mean')
monmean['data'].plot(kind='bar')

⚒ In order to see what is going on there, it is useful to look at the [box plots](https://en.wikipedia.org/wiki/Box_plot) for every month:

In [ ]:
ax = SENSOR.ts.boxplot(column=['data'], by='mon')

⚒ ..and calculate general series statistics

In [ ]:
method.Statistics().execute(SENSOR)

---

## ASSIGNMENT:
Replicate the Multi-year monthly means with data from W_MAGGGIA from the CSV file W_MAGGIA in istsos folder ;-)

https://raw.githubusercontent.com/istSOS/workshop/master/istsos/W_MAGGGIA.csv